# Tuning parameters

In [ ]:
from importlib import reload
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ddn3 import tools, ddn
import parameter_tuning_parallel as parameter_tuning
reload(parameter_tuning)

In [ ]:
# Put the three data in the same folder of this notebook
dat_rna = pd.read_csv("sub.expr.rna.transport", sep='\s+', header=None).to_numpy()
dat_ribo = pd.read_csv("sub.expr.ribo.transport", sep='\s+', header=None).to_numpy()
dat_protein = pd.read_csv("sub.expr.protein.transport", sep='\s+', header=None).to_numpy()

# The number of cores in the computer
cores = 16

Three data pairs

In [ ]:
dat1 = dat_rna
dat2 = dat_ribo

cv_max_deg = np.zeros(dat1.shape[1]) + 200

## Debug

In [ ]:
ratio_val = 0.5
lambda1 = 0.1
lambda2 = 0.05

n_node = dat1.shape[1]
n1 = dat1.shape[0]
n2 = dat2.shape[0]
n1_val = int(n1 * ratio_val)
n1_train = n1 - n1_val
n2_val = int(n2 * ratio_val)
n2_train = n2 - n2_val

mthd = "resi"

msk1 = np.zeros(n1)
msk1[np.random.choice(n1, n1_train, replace=False)] = 1
msk2 = np.zeros(n2)
msk2[np.random.choice(n2, n2_train, replace=False)] = 1

g1_train = tools.standardize_data(dat1[msk1 > 0])
g1_val = tools.standardize_data(dat1[msk1 == 0])
g2_train = tools.standardize_data(dat2[msk2 > 0])
g2_val = tools.standardize_data(dat2[msk2 == 0])

g_beta_est = ddn.ddn_parallel(
    g1_train,
    g2_train,
    lambda1=lambda1,
    lambda2=lambda2,
    mthd=mthd,
    n_process=cores,
)


In [ ]:
reload(parameter_tuning)

In [ ]:
g1_est = tools.get_net_topo_from_mat(g_beta_est[0])
g2_est = tools.get_net_topo_from_mat(g_beta_est[1])

g1_coef = parameter_tuning.calculate_regression(
    g1_train, g1_est * g_beta_est[0], cores=cores, n_max=cv_max_deg
)
g1_coef[np.arange(n_node), np.arange(n_node)] = 0

g2_coef = parameter_tuning.calculate_regression(
    g2_train, g2_est * g_beta_est[1], cores=cores, n_max=cv_max_deg
)
g2_coef[np.arange(n_node), np.arange(n_node)] = 0

rec_ratio1 = np.linalg.norm(
    g1_val @ g1_coef.T - g1_val
) / np.linalg.norm(g1_val)
rec_ratio2 = np.linalg.norm(
    g2_val @ g2_coef.T - g2_val
) / np.linalg.norm(g2_val)
if rec_ratio1 > 2.0 or rec_ratio2 > 2.0:
    print("High rec error")


In [ ]:
np.max(np.abs(g1_coef))

In [ ]:
np.linalg.norm(g1_val @ g1_coef.T - g1_val)

In [ ]:
np.linalg.norm(g1_val)